In [1]:
# 1 - LOAD THE PACKAGES

import pandas as pd
import os
import numpy as np
import sys

In [2]:
# 2 - FILE LOCATION

files = [f for f in os.listdir('.') if os.path.isfile(f)]
files # Take the name of the file

['Technical Test - Data Wrangling.xlsx',
 'CodingAssignment.ipynb',
 '.DS_Store',
 'CodingAssignment.html']

In [3]:
# 3 - LOADING ALL THE FILES INTO A DICTIONARY

dict_test = pd.read_excel('Technical Test - Data Wrangling.xlsx',sheet_name=None)

In [4]:
# 4 - VISUALIZING THE SHEET NAMES

display(dict_test.keys())

dict_keys(['Instructions', 'Data Specification', 'Patient_clinical_data', 'Tissue Sample Metadata', 'Serum Protein data', 'RNA-seq (RPKM)', 'Example report'])

In [5]:
######################################## 5 DATA WRANGLING/SERUM PROTEIN DATA #########################################
# 5.1 - Melting Serum Protein data

SerumProteinDF = pd.melt(dict_test['Serum Protein data'], id_vars=['Patient','Sample'], value_vars=['Serum IL-6 (g/L)', 'Serum IL-6 Receptor (mg/L)'])

In [6]:
# 5.2 - Curating the annotation of IL-6 and IL-6 Receptor

SerumProteinDF['variable'] = SerumProteinDF['variable'].str.replace(" Receptor", "R")
SerumProteinDF['variable'] = SerumProteinDF['variable'].str.replace("-", "")

In [7]:
# 5.3 - Curating the annotation of measurement units

SerumAnno = SerumProteinDF['variable'].str.split(" ",n=2,expand=True)
SerumAnno.columns = ['Material','Gene_Symbol','Result_Units']

SerumAnno['Result_Units'] = SerumAnno['Result_Units'].str.replace("(", "")
SerumAnno['Result_Units'] = SerumAnno['Result_Units'].str.replace(")", "")

In [8]:
# 5.4 - Updating Serum protein data with curated Gene_Symbol and Result_Units 

SerumProteinDF_2 = pd.concat([SerumProteinDF,SerumAnno],axis=1)
mapping = {'Serum':'SERUM'}
SerumProteinDF_2['Material'] = SerumProteinDF_2['Material'].map(mapping)
SerumProteinDF_2.head()

,Patient,Sample,variable,value,Material,Gene_Symbol,Result_Units
0,1001,S-CRC1-A,Serum IL6 (g/L),73.7,SERUM,IL6,g/L
1,1001,S-CRC1-B,Serum IL6 (g/L),67.7,SERUM,IL6,g/L
2,1001,S-CRC1-C,Serum IL6 (g/L),71.3,SERUM,IL6,g/L
3,1002,S-CRC2-A,Serum IL6 (g/L),QNS,SERUM,IL6,g/L
4,1002,S-CRC2-B,Serum IL6 (g/L),33.1,SERUM,IL6,g/L


In [9]:
# 5.5 - Subsetting the columns of interest

cols_interest = ['Sample','Gene_Symbol', 'value','Material','Result_Units']
SerumProteinDF_2[cols_interest].head()

,Sample,Gene_Symbol,value,Material,Result_Units
0,S-CRC1-A,IL6,73.7,SERUM,g/L
1,S-CRC1-B,IL6,67.7,SERUM,g/L
2,S-CRC1-C,IL6,71.3,SERUM,g/L
3,S-CRC2-A,IL6,QNS,SERUM,g/L
4,S-CRC2-B,IL6,33.1,SERUM,g/L


In [10]:
######################################## 6 DATA WRANGLING/ RNA-seq (RPKM) ############################################
# 6.1 - Exploring data
dict_test['RNA-seq (RPKM)'].head()

,GeneID,CRC1N,CRC1T,CRC1M,CRC2N,CRC2T,CRC2M,CRC3N,CRC3T,CRC3M,...,CRC4T,CRC4M,CRC5T,CRC5N,CRC5M,CRC8N,CRC8T,CRC10M,CRC10N,CRC10T
0,ICAM1,4.662774,8.354152,11.863796,5.501785,10.014376,12.980251,6.334321,10.880375,13.788794,...,15.089534,9.005161,6.829498,4.872094,10.344830,8.496029,6.433474,2.482575,9.459477,10.449776
1,IL6,0.000000,0.783566,0.874109,0.000000,2.517168,0.683496,0.108353,0.443711,0.000000,...,0.358051,0.215563,0.132953,0.253945,0.618038,0.280569,0.820672,1.797076,0.000000,0.068515
2,IL6R,12.153740,3.118068,5.728374,12.732372,1.808001,3.215843,10.183599,4.035287,1.545560,...,3.312816,4.748433,2.888519,1.478570,3.644413,3.639156,3.862186,3.701024,3.544909,0.693627
3,VCAM1,4.596351,3.168585,9.143465,4.827985,6.111611,3.900552,10.959019,7.771770,3.187510,...,8.829485,2.862093,7.185174,6.582593,4.299935,7.068253,5.343884,2.688047,7.137615,7.674014
4,SELE,0.077086,1.496338,1.779753,0.099606,2.355495,0.210465,0.042897,1.014963,0.051518,...,0.859222,0.778481,0.326909,0.749328,0.211455,0.767408,0.468175,0.827498,0.996252,0.712139


In [11]:
# 6.1 - Transposing RNA-seq (RPKM) dataframe
RNAseq = dict_test['RNA-seq (RPKM)'].T
new_header = RNAseq.iloc[0]
RNAseq = RNAseq[1:]
RNAseq.columns = new_header
RNAseq.reset_index(inplace=True)
RNAseq.head()

GeneID,index,ICAM1,IL6,IL6R,VCAM1,SELE
0,CRC1N,4.66277,0,12.1537,4.59635,0.0770859
1,CRC1T,8.35415,0.783566,3.11807,3.16859,1.49634
2,CRC1M,11.8638,0.874109,5.72837,9.14346,1.77975
3,CRC2N,5.50178,0,12.7324,4.82799,0.0996059
4,CRC2T,10.0144,2.51717,1.808,6.11161,2.3555


In [12]:
# 6.2 Melt RNAseq dataframe
RNAseq2 = pd.melt(RNAseq, id_vars='index', value_vars=['ICAM1', 'IL6','IL6R','VCAM1','SELE'])
RNAseq2.head()

,index,GeneID,value
0,CRC1N,ICAM1,4.66277
1,CRC1T,ICAM1,8.35415
2,CRC1M,ICAM1,11.8638
3,CRC2N,ICAM1,5.50178
4,CRC2T,ICAM1,10.0144


In [13]:
# 6.3 - Create Material and Result units columns
RNAseq2['Material'] = 'RNA'
RNAseq2['Result_Units'] = 'RPKM'
RNAseq2.head()

,index,GeneID,value,Material,Result_Units
0,CRC1N,ICAM1,4.66277,RNA,RPKM
1,CRC1T,ICAM1,8.35415,RNA,RPKM
2,CRC1M,ICAM1,11.8638,RNA,RPKM
3,CRC2N,ICAM1,5.50178,RNA,RPKM
4,CRC2T,ICAM1,10.0144,RNA,RPKM


In [14]:
# 6.4 - Renaming columns
RNAseq2 = RNAseq2.rename(columns={'index': 'Sample','GeneID':'Gene_Symbol'})
RNAseq2.head()

,Sample,Gene_Symbol,value,Material,Result_Units
0,CRC1N,ICAM1,4.66277,RNA,RPKM
1,CRC1T,ICAM1,8.35415,RNA,RPKM
2,CRC1M,ICAM1,11.8638,RNA,RPKM
3,CRC2N,ICAM1,5.50178,RNA,RPKM
4,CRC2T,ICAM1,10.0144,RNA,RPKM


In [15]:
######################################## 7 Merging RNAseq2 with SerumProteinDF_2 #####################################

RNASerum_df = pd.concat([RNAseq2,SerumProteinDF_2[cols_interest]])
RNASerum_df.head()

,Sample,Gene_Symbol,value,Material,Result_Units
0,CRC1N,ICAM1,4.66277,RNA,RPKM
1,CRC1T,ICAM1,8.35415,RNA,RPKM
2,CRC1M,ICAM1,11.8638,RNA,RPKM
3,CRC2N,ICAM1,5.50178,RNA,RPKM
4,CRC2T,ICAM1,10.0144,RNA,RPKM


In [16]:
############################################ 8 Create PatSample_ID #####################################################
# 8.1 - Renaming Tissue Sample Metadata columns
dict_test['Tissue Sample Metadata'] = dict_test['Tissue Sample Metadata'].rename(columns={'Patient  Number': 'Patient'})

In [17]:
# 8.2 - Select Patient and Sample from Tissue Sample Metadata and SerumProteinDF_2 to create PatSample_id DF

pat_int = ['Patient','Sample']
PatSample_id = pd.concat([dict_test['Tissue Sample Metadata'][pat_int],SerumProteinDF_2[pat_int]])
PatSample_id

,Patient,Sample
0,1001,CRC1N
1,1001,CRC1T
2,1001,CRC1M
3,1002,CRC2N
4,1002,CRC2T
...,...,...
43,2310,S-CRC10-b
44,2310,S-CRC10-c
45,2311,S-CRC11-a
46,2311,S-CRC11-b


In [18]:
############################################ 9 Create RNASerum_Patient DF ############################################
# 9.1 - Merge PatSample_id with RNASerum_df

RNASerum_Patient = pd.merge(PatSample_id,RNASerum_df,on='Sample')
RNASerum_Patient.head()

,Patient,Sample,Gene_Symbol,value,Material,Result_Units
0,1001,CRC1N,ICAM1,4.66277,RNA,RPKM
1,1001,CRC1N,IL6,0,RNA,RPKM
2,1001,CRC1N,IL6R,12.1537,RNA,RPKM
3,1001,CRC1N,VCAM1,4.59635,RNA,RPKM
4,1001,CRC1N,SELE,0.0770859,RNA,RPKM


In [19]:
############################################ 10 RNASerum_Patient_Study DF ##########################################################
# 10.1 - Merge Patient_clinical_data with RNASerum_Patient to create RNASerum_Patient_Study
RNASerum_Patient_Study = pd.merge(dict_test['Patient_clinical_data'],RNASerum_Patient,left_on='Patient  Number',right_on = 'Patient')
RNASerum_Patient_Study.head()

,Study_ID,Patient Number,Sex,Age,Patient,Sample,Gene_Symbol,value,Material,Result_Units
0,RO4532A,1001,M,41.67324,1001,CRC1N,ICAM1,4.66277,RNA,RPKM
1,RO4532A,1001,M,41.67324,1001,CRC1N,IL6,0,RNA,RPKM
2,RO4532A,1001,M,41.67324,1001,CRC1N,IL6R,12.1537,RNA,RPKM
3,RO4532A,1001,M,41.67324,1001,CRC1N,VCAM1,4.59635,RNA,RPKM
4,RO4532A,1001,M,41.67324,1001,CRC1N,SELE,0.0770859,RNA,RPKM


In [20]:
# 10.2 - Create the Unique_Patient_ID
RNASerum_Patient_Study['Unique_Patient_ID'] = RNASerum_Patient_Study['Study_ID'] + '_' + RNASerum_Patient_Study['Patient  Number'].astype('str')
RNASerum_Patient_Study.head()

,Study_ID,Patient Number,Sex,Age,Patient,Sample,Gene_Symbol,value,Material,Result_Units,Unique_Patient_ID
0,RO4532A,1001,M,41.67324,1001,CRC1N,ICAM1,4.66277,RNA,RPKM,RO4532A_1001
1,RO4532A,1001,M,41.67324,1001,CRC1N,IL6,0,RNA,RPKM,RO4532A_1001
2,RO4532A,1001,M,41.67324,1001,CRC1N,IL6R,12.1537,RNA,RPKM,RO4532A_1001
3,RO4532A,1001,M,41.67324,1001,CRC1N,VCAM1,4.59635,RNA,RPKM,RO4532A_1001
4,RO4532A,1001,M,41.67324,1001,CRC1N,SELE,0.0770859,RNA,RPKM,RO4532A_1001


In [21]:
####################### 11 - Creation of a dataframe in the specified format #########################################
# 11.1 - Selecting columns of interest in Tissue Sample Metadata

cols_interest_RNA = ['Patient','Sample','Sample type','Material']
dict_test['Tissue Sample Metadata'][cols_interest_RNA].head()

,Patient,Sample,Sample type,Material
0,1001,CRC1N,Normal,RNA
1,1001,CRC1T,Liver Tumor,RNA
2,1001,CRC1M,Metastic Lung,RNA
3,1002,CRC2N,Normal,RNA
4,1002,CRC2T,Liver Tumor,RNA


In [22]:
# 11.2 - Merge RNASerum_Patient_Study with Tissue Sample Metadata to create the final_df

final_df = pd.merge(RNASerum_Patient_Study, dict_test['Tissue Sample Metadata'][cols_interest_RNA],  how='left', 
                    on=['Patient','Sample','Material'])
final_df.head()

,Study_ID,Patient Number,Sex,Age,Patient,Sample,Gene_Symbol,value,Material,Result_Units,Unique_Patient_ID,Sample type
0,RO4532A,1001,M,41.67324,1001,CRC1N,ICAM1,4.66277,RNA,RPKM,RO4532A_1001,Normal
1,RO4532A,1001,M,41.67324,1001,CRC1N,IL6,0,RNA,RPKM,RO4532A_1001,Normal
2,RO4532A,1001,M,41.67324,1001,CRC1N,IL6R,12.1537,RNA,RPKM,RO4532A_1001,Normal
3,RO4532A,1001,M,41.67324,1001,CRC1N,VCAM1,4.59635,RNA,RPKM,RO4532A_1001,Normal
4,RO4532A,1001,M,41.67324,1001,CRC1N,SELE,0.0770859,RNA,RPKM,RO4532A_1001,Normal


In [23]:
####################################### 12 - Customizing final_Df ####################################################
# 12.1 - Selecting the columns of interest
cols_interest = ['Study_ID','Patient','Unique_Patient_ID','Sex','Age','Sample','Sample type',
                 'Material','Gene_Symbol','value','Result_Units']

final_df_2 = final_df[cols_interest]
final_df_2.head()

,Study_ID,Patient,Unique_Patient_ID,Sex,Age,Sample,Sample type,Material,Gene_Symbol,value,Result_Units
0,RO4532A,1001,RO4532A_1001,M,41.67324,CRC1N,Normal,RNA,ICAM1,4.66277,RPKM
1,RO4532A,1001,RO4532A_1001,M,41.67324,CRC1N,Normal,RNA,IL6,0,RPKM
2,RO4532A,1001,RO4532A_1001,M,41.67324,CRC1N,Normal,RNA,IL6R,12.1537,RPKM
3,RO4532A,1001,RO4532A_1001,M,41.67324,CRC1N,Normal,RNA,VCAM1,4.59635,RPKM
4,RO4532A,1001,RO4532A_1001,M,41.67324,CRC1N,Normal,RNA,SELE,0.0770859,RPKM


In [24]:
# 12.2 - Changing colnames
final_df_2.rename(columns={'Patient': 'Patient_ID','Sample':'Sample_ID','Sample type':'Sample_General_Pathology',
                          'Material':'Material_type','value':'Result'},inplace=True)
final_df_2.head()

/opt/miniconda3/envs/latest_scanpy/lib/python3.8/site-packages/pandas/core/frame.py:4293: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Study_ID,Patient_ID,Unique_Patient_ID,Sex,Age,Sample_ID,Sample_General_Pathology,Material_type,Gene_Symbol,Result,Result_Units
0,RO4532A,1001,RO4532A_1001,M,41.67324,CRC1N,Normal,RNA,ICAM1,4.66277,RPKM
1,RO4532A,1001,RO4532A_1001,M,41.67324,CRC1N,Normal,RNA,IL6,0,RPKM
2,RO4532A,1001,RO4532A_1001,M,41.67324,CRC1N,Normal,RNA,IL6R,12.1537,RPKM
3,RO4532A,1001,RO4532A_1001,M,41.67324,CRC1N,Normal,RNA,VCAM1,4.59635,RPKM
4,RO4532A,1001,RO4532A_1001,M,41.67324,CRC1N,Normal,RNA,SELE,0.0770859,RPKM


In [25]:
######################################### 13 - Modifying data types ##################################################
# 13.1 Transforming Result into a numeric variable
final_df_2['Result'] = pd.to_numeric(final_df_2['Result'], errors='coerce')

<ipython-input-25-4a78f73ca3f9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_2['Result'] = pd.to_numeric(final_df_2['Result'], errors='coerce')


In [26]:
# 13.2 Transforming Age into a numeric variable
final_df_2['Age'] = final_df_2['Age'].round().astype('int')

<ipython-input-26-8b5440c36856>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_2['Age'] = final_df_2['Age'].round().astype('int')


In [27]:
######################################### 14 - Modifying feature entries #############################################
# 14.1 - Changing the sex entries
mapping = {'M':'MALE', 'F':'FEMALE'}
final_df_2['Sex'] = final_df_2['Sex'].map(mapping)

<ipython-input-27-235fc1e44a11>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_2['Sex'] = final_df_2['Sex'].map(mapping)


In [28]:
# 14.2 - Changing Sample_General_Pathology entries
mapping2 = {'Normal':'NORMAL', 'Liver Tumor':'PRIMARY','Metastic Lung':'METASTATIC'}
final_df_2['Sample_General_Pathology'] = final_df_2['Sample_General_Pathology'].map(mapping2)
final_df_2['Sample_General_Pathology'].fillna('NA',inplace = True)

<ipython-input-28-35ab97279f9a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_2['Sample_General_Pathology'] = final_df_2['Sample_General_Pathology'].map(mapping2)
/opt/miniconda3/envs/latest_scanpy/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [29]:
# 14.3 - Curate Result_Units. Transform entries with mg/L as g/L

new_results = []
for res,units in zip(final_df_2['Result'],final_df_2['Result_Units']):
    if units == 'mg/L':
        new_results.append(res/1000)
    elif units == 'g/L':
        new_results.append(res)
    elif units == 'RPKM':
        new_results.append(res)

final_df_2['Result'] = new_results

<ipython-input-29-5a5d28e4070b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_2['Result'] = new_results


In [30]:
# 14.4 - Change the mg/L to g/L in Result_Units
mapping3 = {'mg/L':'g/L','RPKM':'RPKM','g/L':'g/L'}
final_df_2['Result_Units'] = final_df_2['Result_Units'].map(mapping3)

<ipython-input-30-25ee5e0f7c3d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_2['Result_Units'] = final_df_2['Result_Units'].map(mapping3)


In [31]:
# 14.5 - Creating the columns status

status = []
for result in final_df_2['Result']:
    if np.isnan(result):
        status.append('NOT DONE')
    else:
        status.append('NA')
final_df_2['Status'] = status

In [32]:
# 14.6 - Capitalize Sample_ID
final_df_2['Sample_ID'] = final_df_2['Sample_ID'].str.upper()

In [33]:
# 14.7 - Print the result
final_df_2

,Study_ID,Patient_ID,Unique_Patient_ID,Sex,Age,Sample_ID,Sample_General_Pathology,Material_type,Gene_Symbol,Result,Result_Units,Status
0,RO4532A,1001,RO4532A_1001,MALE,42,CRC1N,NORMAL,RNA,ICAM1,4.662774,RPKM,NA
1,RO4532A,1001,RO4532A_1001,MALE,42,CRC1N,NORMAL,RNA,IL6,0.000000,RPKM,NA
2,RO4532A,1001,RO4532A_1001,MALE,42,CRC1N,NORMAL,RNA,IL6R,12.153740,RPKM,NA
3,RO4532A,1001,RO4532A_1001,MALE,42,CRC1N,NORMAL,RNA,VCAM1,4.596351,RPKM,NA
4,RO4532A,1001,RO4532A_1001,MALE,42,CRC1N,NORMAL,RNA,SELE,0.077086,RPKM,NA
...,...,...,...,...,...,...,...,...,...,...,...,...
191,RO4532A,2311,RO4532A_2311,MALE,75,S-CRC11-B,NA,SERUM,IL6R,0.003010,g/L,NA
192,RO4532A,2311,RO4532A_2311,MALE,75,S-CRC11-C,NA,SERUM,IL6,31.400000,g/L,NA
193,RO4532A,2311,RO4532A_2311,MALE,75,S-CRC11-C,NA,SERUM,IL6R,0.002940,g/L,NA
194,RO4532A,2311,RO4532A_2311,MALE,75,S-CRC11-C,NA,SERUM,IL6,31.400000,g/L,NA


In [34]:
################################################ 15 - FINAL REMARKS ##################################################
# 15.1 - Version of packages used
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

pandas==1.1.1
numpy==1.19.1


In [35]:
# 15.2 - Python version
print(sys.version)

3.8.10 | packaged by conda-forge | (default, May 10 2021, 22:58:09) 
[Clang 11.1.0 ]


In [36]:
################################################ 16 - OUTPUT TO CSV FILE ##############################################
final_df_2.to_csv('TechnicalTest_result.csv')